In [ ]:
import pandas as pd
import mysql.connector

# Load the Excel file
file_path = r"file_path"
sheet_data = pd.read_excel(file_path, sheet_name='intercropped_crops')

# Connect to MySQL database
db_connection = mysql.connector.connect(
    host='localhost',  # Replace with your host (e.g., 'localhost')
    user='root',  # Replace with your MySQL username
    password='*****',  # Replace with your MySQL password
    database='*****'  # Replace with your database name
)

cursor = db_connection.cursor()

# Iterate through each row of the Excel data
for index, row in sheet_data.iterrows():
    last_name = row['last_name']
    first_name = row['first_name']
    intercropped_crops = row['intercropped_crops'] if pd.notnull(row['intercropped_crops']) else None

    # Step 1: Get farmer_id using last_name and first_name
    cursor.execute("""
        SELECT farmer_id FROM farmer 
        WHERE last_name = %s AND first_name = %s
    """, (last_name, first_name))

    result = cursor.fetchone()

    if result:
        farmer_id = result[0]

        # Step 2: Insert data into intercropping_data
        cursor.execute("""
            INSERT INTO intercropping_data (farmer_id, intercropped_crops)
            VALUES (%s, %s)
        """, (farmer_id, intercropped_crops))
    else:
        print(f"No match found for {first_name} {last_name}. Skipping row {index + 1}.")

# Commit the changes and close the connection
db_connection.commit()
cursor.close()
db_connection.close()

print("Data import completed.")
